In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct credentials from service account key file
credentials = service_account.Credentials.from_service_account_file(
    "C:\\Users\\user\\Downloads\\hlai27_bq (1)\\hlai27-isom676-srvacct.json")

# Construct a BigQuery client object
client = bigquery.Client(credentials=credentials)

In [ ]:
import duckdb

# 🔹 Define file paths for pre-sampled data
sampled_transactions_parquet = "C:\\Users\\user\\Downloads\\ACSE Data\\sampled_transactions.parquet"
cleaned_products_parquet = "C:\\Users\\user\\Downloads\\ACSE Data\\cleaned_products.parquet"

# 🔹 Connect to DuckDB
conn = duckdb.connect()

# 🔹 Execute Query for Product Grouping
query = f"""
WITH product_sales AS (
  SELECT 
    p.prod_id,
    p.prod_category,
    p.prod_subcategory,
    p.prod_type,
    COUNT(DISTINCT t.trans_id) AS total_transactions,
    SUM(t.sales_amt) AS total_revenue,
    SUM(t.sales_qty) AS total_units_sold,
    SUM(t.sales_amt) / NULLIF(SUM(t.sales_qty), 0) AS avg_unit_price
  FROM 
    read_parquet('{sampled_transactions_parquet}') t
  JOIN 
    read_parquet('{cleaned_products_parquet}') p 
  ON 
    t.prod_id = p.prod_id
  GROUP BY 
    p.prod_id, p.prod_category, p.prod_subcategory, p.prod_type
),

promotion_analysis AS (
  SELECT
    t.prod_id,
    COUNT(DISTINCT t.trans_id) AS promo_transactions,
    SUM(t.sales_amt) AS promo_revenue,
    (COUNT(DISTINCT t.trans_id) * 100.0) / NULLIF(ps.total_transactions, 0) AS promo_percentage
  FROM 
    read_parquet('{sampled_transactions_parquet}') t
  JOIN 
    product_sales ps 
  ON 
    t.prod_id = ps.prod_id
  WHERE 
    (t.sales_amt / NULLIF(t.sales_qty, 0)) < 0.9 * ps.avg_unit_price
  GROUP BY 
    t.prod_id, ps.total_transactions
),

traffic_drivers AS (
  SELECT 
    prod_id
  FROM read_parquet('{sampled_transactions_parquet}')
  GROUP BY prod_id
  ORDER BY COUNT(*) DESC
  LIMIT 100  
)

-- **Final Selection of Required Product Categories**
SELECT 
  product_grouping,
  COUNT(*) AS product_count
FROM (
  SELECT 
    ps.prod_id, 
    CASE 
      -- 🚀 **Traffic Driver (Highest Priority)**
      WHEN ps.prod_id IN (SELECT prod_id FROM traffic_drivers) 
      THEN 'Traffic Driver'

      -- 🔹 **Always Promoted**
      WHEN pa.promo_percentage > 50 THEN 'Always Promoted'

      -- 🔹 **Occasionally Promoted**
      WHEN pa.promo_percentage BETWEEN 10 AND 50 THEN 'Occasionally Promoted'

      -- 🔹 **Seldom/Never Promoted**
      ELSE 'Seldom/Never Promoted'
    END AS product_grouping
  FROM 
    product_sales ps
  LEFT JOIN 
    promotion_analysis pa ON ps.prod_id = pa.prod_id
) grouped_products
WHERE product_grouping IN ('Seldom/Never Promoted', 'Traffic Driver', 'Occasionally Promoted', 'Always Promoted')
GROUP BY 
  product_grouping
ORDER BY 
  product_count DESC;
"""

try:
    # 🔹 Run Query and Fetch Data
    result_df = conn.execute(query).fetchdf()

    # 🔹 Display Results
    print("\n📊 Product Grouping Counts:")
    print("===========================================")
    print("{:<25} {:<15}".format("Product Grouping", "Product Count"))
    print("===========================================")

    for index, row in result_df.iterrows():
        print("{:<25} {:<15}".format(row["product_grouping"], row["product_count"]))

    # Optional: Display as a DataFrame
    print("\n🔍 Data Preview:")
    print(result_df)

except Exception as e:
    print(f"❌ Error occurred during query execution: {e}")


In [ ]:
from google.cloud import bigquery
import matplotlib.pyplot as plt
import pandas as pd


gcp_credentials_path = "C:\\Users\\user\\Downloads\\hlai27_bq (1)\\hlai27-isom676-srvacct.json"


client = bigquery.Client.from_service_account_json(gcp_credentials_path)


query = """
WITH product_sales AS (
  SELECT 
    p.prod_id,
    p.prod_category,
    COUNT(t.trans_id) AS total_transactions
  FROM 
    `msba-emory.isom676_machine_learning.transactions` t
  JOIN 
    `msba-emory.isom676_machine_learning.products` p 
  ON 
    t.prod_id = p.prod_id
  GROUP BY 
    p.prod_id, p.prod_category
),

traffic_drivers AS (
  SELECT 
    prod_id,
    prod_category,
    total_transactions,
    ROW_NUMBER() OVER (ORDER BY total_transactions DESC) AS rank
  FROM 
    product_sales
)

-- 計算 Traffic Driver 產品的品類數量
SELECT 
  prod_category,
  COUNT(prod_id) AS product_count
FROM 
  traffic_drivers
WHERE 
  rank <= 100
GROUP BY
  prod_category
ORDER BY
  product_count DESC;
"""


query_job = client.query(query)
df = query_job.to_dataframe()


print(df.head())


plt.figure(figsize=(12, 6))
plt.bar(df['prod_category'], df['product_count'], color='deepskyblue')
plt.title('Traffic Driver Products by Category')
plt.xlabel('Product Category')
plt.ylabel('Number of Products')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()


In [ ]:
from google.cloud import bigquery
import matplotlib.pyplot as plt
import pandas as pd


gcp_credentials_path = "C:\\Users\\user\\Downloads\\hlai27_bq (1)\\hlai27-isom676-srvacct.json"

client = bigquery.Client.from_service_account_json(gcp_credentials_path)

query = """
WITH product_sales AS (
  SELECT 
    p.prod_id,
    p.prod_category,
    COUNT(DISTINCT t.trans_id) AS total_transactions,
    SUM(t.sales_amt) AS total_revenue,
    SUM(t.sales_qty) AS total_units_sold,
    SAFE_DIVIDE(SUM(t.sales_amt), NULLIF(SUM(t.sales_qty), 0)) AS avg_unit_price
  FROM 
    `msba-emory.isom676_machine_learning.transactions` t
  JOIN 
    `msba-emory.isom676_machine_learning.products` p 
  ON 
    t.prod_id = p.prod_id
  GROUP BY 
    p.prod_id, p.prod_category
),

promotion_analysis AS (
  SELECT
    t.prod_id,
    COUNT(DISTINCT t.trans_id) AS promo_transactions,
    SAFE_DIVIDE(COUNT(DISTINCT t.trans_id), NULLIF(ps.total_transactions, 0)) * 100 AS promo_percentage
  FROM 
    `msba-emory.isom676_machine_learning.transactions` t
  JOIN 
    product_sales ps 
  ON 
    t.prod_id = ps.prod_id
  WHERE 
    (t.sales_amt / NULLIF(t.sales_qty, 0)) < 0.9 * ps.avg_unit_price
  GROUP BY 
    t.prod_id, ps.total_transactions
)

-- 計算 Always Promoted 產品的品類數量
SELECT 
  p.prod_category,
  COUNT(p.prod_id) AS product_count
FROM 
  promotion_analysis pa
JOIN 
  `msba-emory.isom676_machine_learning.products` p 
ON 
  pa.prod_id = p.prod_id
WHERE 
  pa.promo_percentage > 50
GROUP BY
  p.prod_category
ORDER BY
  product_count DESC;
"""

query_job = client.query(query)
df = query_job.to_dataframe()

print(df.head())

plt.figure(figsize=(12, 6))
plt.bar(df['prod_category'], df['product_count'], color='tomato')
plt.title('Always Promoted Products by Category')
plt.xlabel('Product Category')
plt.ylabel('Number of Products')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()


In [ ]:
from google.cloud import bigquery
import matplotlib.pyplot as plt
import pandas as pd

gcp_credentials_path = "C:\\Users\\user\\Downloads\\hlai27_bq (1)\\hlai27-isom676-srvacct.json"


client = bigquery.Client.from_service_account_json(gcp_credentials_path)


query = """
WITH product_sales AS (
  SELECT 
    p.prod_id,
    p.prod_category,
    COUNT(DISTINCT t.trans_id) AS total_transactions,
    SUM(t.sales_amt) AS total_revenue,
    SUM(t.sales_qty) AS total_units_sold,
    SAFE_DIVIDE(SUM(t.sales_amt), NULLIF(SUM(t.sales_qty), 0)) AS avg_unit_price
  FROM 
    `msba-emory.isom676_machine_learning.transactions` t
  JOIN 
    `msba-emory.isom676_machine_learning.products` p 
  ON 
    t.prod_id = p.prod_id
  GROUP BY 
    p.prod_id, p.prod_category
),

promotion_analysis AS (
  SELECT
    t.prod_id,
    COUNT(DISTINCT t.trans_id) AS promo_transactions,
    SAFE_DIVIDE(COUNT(DISTINCT t.trans_id), NULLIF(ps.total_transactions, 0)) * 100 AS promo_percentage
  FROM 
    `msba-emory.isom676_machine_learning.transactions` t
  JOIN 
    product_sales ps 
  ON 
    t.prod_id = ps.prod_id
  WHERE 
    (t.sales_amt / NULLIF(t.sales_qty, 0)) < 0.9 * ps.avg_unit_price
  GROUP BY 
    t.prod_id, ps.total_transactions
)

-- 計算 Seldom/Never Promoted 產品的品類數量
SELECT 
  p.prod_category,
  COUNT(p.prod_id) AS product_count
FROM 
  promotion_analysis pa
JOIN 
  `msba-emory.isom676_machine_learning.products` p 
ON 
  pa.prod_id = p.prod_id
WHERE 
  pa.promo_percentage < 10  -- 🔹 促銷比例 < 10% 表示極少促銷
GROUP BY
  p.prod_category
ORDER BY
  product_count DESC;
"""


query_job = client.query(query)
df = query_job.to_dataframe()


print(df.head())


plt.figure(figsize=(12, 6))
plt.bar(df['prod_category'], df['product_count'], color='royalblue')
plt.title('Seldom/Never Promoted Products by Category')
plt.xlabel('Product Category')
plt.ylabel('Number of Products')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()
